This notebook is designed to help me build Magic: The Gathering decks using external data from the website "Scryfall," which tracks datapoints on all cards, such as set, rarity, price, and more.

In building this, I hope to automate the process of budgeting for a new deck by calling Scryfall's API to get me prices.

I am also evaluating using a task scheduler to execute this process on a set schedule. This way, I could evaluate when a card price seems to be fluctuating, allowing me to make a decision about when I want to purchase it.

In [1]:
import requests as requests

In [2]:
# An initial exploration into Scryfall's "bulk" repository.

url = 'https://api.scryfall.com/bulk-data'
headers = {'Content-type': 'application/json'}
r = requests.get(url, headers=headers)
r.text

'{"object":"list","has_more":false,"data":[{"object":"bulk_data","id":"27bf3214-1271-490b-bdfe-c0be6c23d02e","type":"oracle_cards","updated_at":"2023-03-11T10:02:32.225+00:00","uri":"https://api.scryfall.com/bulk-data/27bf3214-1271-490b-bdfe-c0be6c23d02e","name":"Oracle Cards","description":"A JSON file containing one Scryfall card object for each Oracle ID on Scryfall. The chosen sets for the cards are an attempt to return the most up-to-date recognizable version of the card.","size":112181077,"download_uri":"https://data.scryfall.io/oracle-cards/oracle-cards-20230311100232.json","content_type":"application/json","content_encoding":"gzip"},{"object":"bulk_data","id":"6bbcf976-6369-4401-88fc-3a9e4984c305","type":"unique_artwork","updated_at":"2023-03-11T10:03:10.078+00:00","uri":"https://api.scryfall.com/bulk-data/6bbcf976-6369-4401-88fc-3a9e4984c305","name":"Unique Artwork","description":"A JSON file of Scryfall card objects that together contain all unique artworks. The chosen cards 

In [3]:
# Looks good!
r.status_code

200

This is where the magic (hah) happens. Now that we have a connection to the API, we can write these card entries to a Pandas dataframe. First, we load in our Excel spreadsheet. Then we us a "for" loop to iterate over the names in the spreadsheet.

Once we have names, we create a blank "prices" list, then generate URLs based on card name--this is accomplished using an F-string.

In the same "for" loop, we also query the API, retrieve a response in JSON, and append the 'price' column entry for that card.

Last, we output to a Pandas dataframe and check the results. Looking good!

In [6]:
import pandas as pd
import openpyxl

# Load the Excel file and select the active worksheet
wb = openpyxl.load_workbook('cards.xlsx')
ws = wb.active

# Create a list of card names from the Excel worksheet
card_names = [row[0] for row in ws.values]

# Create a list of prices for each card name using the Scryfall API
prices = []
for card_name in card_names:
    url = f"https://api.scryfall.com/cards/named?fuzzy={card_name}"
    response = requests.get(url)
    data = response.json()
    price = data['prices']['usd']
    prices.append(price)

# Create a Pandas dataframe with the card names and prices
df = pd.DataFrame({'Card Name': card_names, 'Price': prices})

# Print the resulting dataframe
print(df.head())


                    Card Name Price
0                   card_name  0.23
1              Adarkar Wastes  3.93
2   Aminatou, the Fateshifter  3.11
3  Approach of the Second Sun  3.49
4               Arcane Signet  0.95


In [36]:
print(card_data)

{'object': 'card', 'id': '5c1f3f52-cb9b-4b2a-bb02-6175897ae76e', 'oracle_id': '94a3b2af-0741-48c5-b827-6ff529bafae3', 'multiverse_ids': [74237], 'tcgplayer_id': 37906, 'cardmarket_id': 14870, 'name': 'Our Market Research Shows That Players Like Really Long Card Names So We Made this Card to Have the Absolute Longest Card Name Ever Elemental', 'lang': 'en', 'released_at': '2004-11-19', 'uri': 'https://api.scryfall.com/cards/5c1f3f52-cb9b-4b2a-bb02-6175897ae76e', 'scryfall_uri': 'https://scryfall.com/card/unh/107/our-market-research-shows-that-players-like-really-long-card-names-so-we-made-this-card-to-have-the-absolute-longest-card-name-ever-elemental?utm_source=api', 'layout': 'normal', 'highres_image': True, 'image_status': 'highres_scan', 'image_uris': {'small': 'https://cards.scryfall.io/small/front/5/c/5c1f3f52-cb9b-4b2a-bb02-6175897ae76e.jpg?1562488399', 'normal': 'https://cards.scryfall.io/normal/front/5/c/5c1f3f52-cb9b-4b2a-bb02-6175897ae76e.jpg?1562488399', 'large': 'https://ca

In [7]:
df.head(10)

,Card Name,Price
0,card_name,0.23
1,Adarkar Wastes,3.93
2,"Aminatou, the Fateshifter",3.11
3,Approach of the Second Sun,3.49
4,Arcane Signet,0.95
5,"Atraxa, Praetors' Voice",75.54
6,Auramancer,0.09
7,Binding the Old Gods,0.60
8,Birth of the Imperium,0.20
9,Bountiful Promenade,4.20


In [8]:
# Oops, looks like the index was queried as well. Dropping this entry.

df1 = df.drop([0])
df1.head()

,Card Name,Price
1,Adarkar Wastes,3.93
2,"Aminatou, the Fateshifter",3.11
3,Approach of the Second Sun,3.49
4,Arcane Signet,0.95
5,"Atraxa, Praetors' Voice",75.54


In [9]:
# Checking for working directory to see where results will be exported.
import os
print(os.getcwd())

C:\Users\twiss\Documents


In [10]:
# Export the complete CSV to a directory for reference. Separate using tabs to create separate columns.
df1.to_csv('scryfall.csv', sep='\t', index=False)